In [ ]:
import pandas as pd
import os
from pathlib import Path

def convert_excel_to_csv(folder_path):
    """
    Convert all Excel files in a folder to CSV and show their previews.
    
    Args:
        folder_path (str): Path to the folder containing Excel files
    """
    # Convert folder path to Path object
    folder = Path(folder_path)
    
    # Loop through all files in the folder
    for file in folder.glob('*.xlsx'):
        try:
            # Read the Excel file
            df = pd.read_excel(file)
            
            # Create CSV filename (same name, different extension)
            csv_path = file.with_suffix('.csv')
            
            # Save as CSV
            df.to_csv(csv_path, index=False)
            
            # Print information about the conversion
            print(f"\nConverted: {file.name} → {csv_path.name}")
            print(f"Shape: {df.shape}")
            print("\nPreview:")
            print(df.head())
            print("\n" + "="*50)
            
        except Exception as e:
            print(f"Error processing {file.name}: {str(e)}")

# Example usage:
# convert_excel_to_csv('data')